## Create a model and export it as ONNX

In [ ]:
import torch

batch_size = 1
device = 'cuda'
model = torch.hub.load('ultralytics/yolov5', 'yolov5s').to(device)

After training...

In [ ]:
torch.save(model.state_dict(), './models/obj.pt')

In [ ]:
dummy_input = torch.randn(batch_size, 3, 256, 256).to(device)

with torch.no_grad():
    print(model(dummy_input).shape)

torch.onnx.export(model, dummy_input, './models/obj.onnx')

## Convert the ONNX model to TensorRT engine

In [ ]:
# Restart notebook kernel
import os
os._exit(00)

- Convert to FP32 engine

In [ ]:
!trtexec --onnx=./models/obj.onnx --saveEngine=./models/obj_32.engine 

- Convert to FP16 engine

In [ ]:
!trtexec --onnx=./models/obj.onnx --saveEngine=./models/obj_16.engine --fp16 

## Benchmark

In [ ]:
from benchmark import NativeTorchBenchmark, TensorRTBehcnmark, TorchScriptBenchmark
import numpy as np
import torch

n_infers = 100
batch_size = 1
input_image = np.random.normal(size=[batch_size, 3, 256, 256])

- Native PyTorch

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

pt_bm = NativeTorchBenchmark(n_infers=n_infers,
                             batch_size=batch_size,
                             samples=input_image,
                             model_arch=model,
                             model_ckpt='./models/obj.pt')

In [ ]:
pt_bm.benchmark()

- Torch Script

not support

- TensorRT FP32

In [ ]:
trt_bm_fp32 = TensorRTBehcnmark(n_infers=n_infers,
                                batch_size=batch_size,
                                samples=input_image,
                                engine_path='./models/obj_32.engine')

In [ ]:
trt_bm_fp32.benchmark()

- TensorRT FP16

In [ ]:
trt_bm_fp16 = TensorRTBehcnmark(n_infers=n_infers,
                                batch_size=batch_size,
                                samples=input_image,
                                engine_path='./models/obj_16.engine')

In [ ]:
trt_bm_fp16.benchmark()